In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('/home/ubuntu/fastai/')
from fastai.imports import *

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
import warnings
warnings.simplefilter('ignore', UserWarning)
warnings.simplefilter('ignore', DeprecationWarning)

In [3]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [4]:
PATH = '/home/ubuntu/data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'

TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

nietzsche.txt  trn/  val/


In [5]:
%ls {PATH}trn

trn.txt


In [6]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(945, 55, 1, 484446)

In [7]:
TEXT.vocab.stoi.items()

dict_items([('<unk>', 0), ('<pad>', 1), (' ', 2), ('e', 3), ('t', 4), ('i', 5), ('a', 6), ('o', 7), ('n', 8), ('s', 9), ('r', 10), ('h', 11), ('l', 12), ('d', 13), ('c', 14), ('u', 15), ('f', 16), ('m', 17), ('p', 18), ('g', 19), (',', 20), ('y', 21), ('w', 22), ('b', 23), ('v', 24), ('-', 25), ('.', 26), ('"', 27), ('k', 28), ('x', 29), (';', 30), (':', 31), ('q', 32), ('j', 33), ('!', 34), ('?', 35), ('(', 36), (')', 37), ("'", 38), ('z', 39), ('1', 40), ('2', 41), ('=', 42), ('_', 43), ('3', 44), ('[', 45), (']', 46), ('5', 47), ('4', 48), ('6', 49), ('8', 50), ('7', 51), ('9', 52), ('0', 53), ('ä', 54), ('æ', 0), ('ë', 0), ('é', 0), ('<eos>', 0)])

In [8]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [9]:
m = CharSeqStatefulRnn(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [10]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.869916   1.850201  
    1      1.6978     1.701003                               
    2      1.620618   1.634215                               
    3      1.557539   1.594983                               



[1.5949827439796467]

In [11]:
set_lrs(opt, 1e-4)

fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.483153   1.556302  
    1      1.47462    1.550348                               
    2      1.478654   1.546873                               
    3      1.476688   1.54328                                



[1.5432797363190534]

In [16]:
# From the pytorch source

def RNNCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    return F.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh, b_hh))

In [17]:
class CharSeqStatefulRnn2(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNNCell(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp = []
        o = self.h
        for c in cs: 
            o = self.rnn(self.e(c), o)
            outp.append(o)
        outp = self.l_out(torch.stack(outp))
        self.h = repackage_var(o)
        return F.log_softmax(outp, dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [18]:
m = CharSeqStatefulRnn2(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [20]:
# From the pytorch source code - for reference

def GRUCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    gi = F.linear(input, w_ih, b_ih)
    gh = F.linear(hidden, w_hh, b_hh)
    i_r, i_i, i_n = gi.chunk(3, 1)
    h_r, h_i, h_n = gh.chunk(3, 1)

    resetgate = F.sigmoid(i_r + h_r)
    inputgate = F.sigmoid(i_i + h_i)
    newgate = F.tanh(i_n + resetgate * h_n)
    return newgate + inputgate * (hidden - newgate)

In [22]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [23]:
m = CharSeqStatefulGRU(md.nt, n_fac, 512).cuda()

opt = optim.Adam(m.parameters(), 1e-3)

In [24]:
fit(m, md, 6, opt, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.75718    1.735421  
    1      1.569917   1.58765                                
    2      1.491203   1.523672                               
    3      1.438345   1.48699                                
    4      1.401069   1.471094                               
    5      1.374436   1.462502                               



[1.4625024508255033]

In [25]:
set_lrs(opt, 1e-4)

In [26]:
fit(m, md, 3, opt, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.277462   1.427466  
    1      1.280148   1.423905                               
    2      1.281089   1.421479                               



[1.4214793925818343]

In [27]:
from fastai import sgdr

n_hidden=512

In [28]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [29]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [30]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [31]:
fit(m, md, 2, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.803206   1.72403   
    1      1.696772   1.622173                              



[1.6221729931063378]

In [32]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss                              
    0      1.514841   1.475145  
    1      1.574374   1.504719                              
    2      1.43657    1.420014                              
    3      1.591089   1.530053                              
    4      1.520575   1.470733                              
    5      1.429645   1.406932                              
    6      1.363477   1.372792                              
    7      1.580188   1.524284                              
    8      1.547492   1.49386                               
    9      1.508749   1.471797                              
    10     1.475558   1.440631                              
    11     1.424963   1.403219                              
    12     1.366429   1.372696                              
    13     1.328982   1.344253                              
    14     1.292904   1.333059                              



[1.3330591411974568]

In [33]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss                              
    0      1.281877   1.331015  
    1      1.276795   1.329036                              
    2      1.280829   1.328249                              
    3      1.277025   1.324984                              
    4      1.266703   1.322078                              
    5      1.262509   1.32108                               
    6      1.259342   1.320134                              
    7      1.263304   1.318831                              
    8      1.259897   1.31662                               
    9      1.253291   1.315739                              
    10     1.240744   1.313392                              
    11     1.231264   1.313849                              
    12     1.23226    1.312939                              
    13     1.225177   1.31297                               
    14     1.22058    1.312448                              
    15     1.226616   1.31285                       

[1.3751954961424837]

In [34]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [35]:
get_next('for thos')

'e'

In [36]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [37]:
print(get_next_n('for thos', 400))

for those arrow than the manners are inventions as a privileged to suchfrenchmony, free spirits, andhere, and through that is no ones),--the sparer whan ohd taste and sacrifice" and stand. one may now now difference for you? for it own.--such a approval or-interprequestinism--as wewe, pelated). he experience2. almost again and conflictisnes, as a satisfacious (they should inheritand possessly of revoice, 
